In [1]:
import sys
print(sys.executable)
import ipykernel
print(ipykernel.__version__)

/home/aidev/patchtst_timeseries/.venv/bin/python
6.31.0


In [ ]:
%%capture
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
import logging
import os
from ticker_data import get_ticker
# Set both HTTP and HTTPS proxies

predict_ticker = 'AAPL'
tickers = [predict_ticker, 'SPY']
_start, _end = '2021-06-01', '2024-12-30'
ticker_data = {t: get_ticker(t, start=_start, end=_end) for t in tickers}

# 2. BASIS-BERECHNUNG (Log-Returns & Volumen)
# Wir berechnen hier nur die Roh-Features, die keine Roll-Fenster nutzen
df_close = pd.DataFrame({t: ticker_data[t]['Close'] for t in tickers})
df_volume = pd.DataFrame({t: ticker_data[t]['Volume'] for t in tickers})

df_close_log = np.log(df_close / df_close.shift(1))
df_vol_log = np.log(df_volume).diff()

# Echte AAPL-Preise für P&L-Berechnung
aapl_open = ticker_data[predict_ticker]['Open']
aapl_close = ticker_data[predict_ticker]['Close']

# 3. DIE SCHLEIFE
test_dates = df_close_log['2024-01-01':].index
portfolio_value = 10000.0
current_pos = 0
position_return = 0
entry_price = None
THRESHOLD = 0.008
FEE = 0.0005
log_file = 'from_trained_test.csv'
STOPLOSS_THRESHOLD = -0.01
SIGNAL_TRIGGER_STOPLOSS = -2
SIGNAL_TRIGGER_TP = 2
# Log-Header
with open(log_file, 'w') as f:
    f.write("date,pred_momentum,signal,in_market,trade_return,portfolio_value,position_return\n")

#load model
nf = NeuralForecast.load(path='./checkpoints/patchtst_momentum_model_30days/')
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR) 
logging.getLogger("neuralforecast").setLevel(logging.ERROR) 
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR) 
trade = None
for i, today in enumerate(test_dates):
    # --- DATEN-FENSTER FÜR HEUTE ERSTELLEN ---
    df_list = []
    for t in tickers:
        # Nur Daten bis 'today'
        #mask = df_close_log.index <= today 
        # PRICE: keep only up to today, then last 30 points
        s_price = df_close_log.loc[df_close_log.index <= today, t].tail(30)        
        
        # Preis-Kanäle
        df_list.append(pd.DataFrame({
            'ds': s_price.index,
            'unique_id': f'{t}_price',
            'y': s_price.values
        }))
        s_vol = df_vol_log.loc[df_vol_log.index <= today, t].tail(30)
        # Volumen-Kanäle
        df_list.append(pd.DataFrame({
            'ds': s_price.index,
            'unique_id': f'{t}_vol',
            'y': s_price.values
        }))

    df_step = pd.concat(df_list).dropna()
        
    # Prediction für den nächsten Tag
    forecast = nf.predict(df=df_step)
    #print(forecast)
    # Wir nehmen den ersten Tag des Horizonts (t+1)
    pred_momentum1 = forecast.query(f"unique_id == '{predict_ticker}_price'").iloc[0]['PatchTST']
    pred_momentum2 = forecast.query(f"unique_id == '{predict_ticker}_price'").iloc[1]['PatchTST']
    #print(forecast)
    # Signal Logik
    signal = 0
    pred_momentum = pred_momentum1 + pred_momentum2
    if pred_momentum > THRESHOLD: signal = 1
    elif pred_momentum < -THRESHOLD: signal = -1
    
    # --- ABRECHNUNG MIT ECHTEN PREISEN ---
    trade_return = 0.0

    if current_pos != 0:
        close_today = aapl_close.loc[today]
        position_return = current_pos * (close_today - entry_price) / entry_price    
        #STOPLOSS
        if position_return < STOPLOSS_THRESHOLD:
            signal  = SIGNAL_TRIGGER_STOPLOSS
    else:
        position_return = 0
    
    this_positon_return = position_return #needed for logging    
    day_current_signal = signal #needed for logging    
    day_current_pos = current_pos
    portfolio_value *= (1 + this_positon_return)
    #if signal != 0: # keep position open if there is no signal
    if True:    
        if signal != current_pos:
            # 1. Bestehende Position schließen zum heutigen Close
            if current_pos != 0 and entry_price is not None:
                #close_today = aapl_close.loc[today]
                #trade_return = current_pos * (close_today - entry_price) / entry_price
                #portfolio_value *= (1 + trade_return)
                position_return = 0
                signal = 0
                current_pos = 0
            # 2. Kosten bei Signalwechsel (Roundtrip-Gebühr)
            portfolio_value *= (1 - FEE)
            
            # 3. Neue Position eröffnen zum Close of today !
            if signal != 0 and i + 1 < len(test_dates):
                next_day = test_dates[i + 1]
                entry_price = aapl_open.loc[next_day]
            #    entry_price = aapl_close.loc[today]     
                current_pos = signal     
            else:
                entry_price = None
                        
    
    print(f"{today.date()} | Pred: {pred_momentum:.5f} | Signal: {signal} | "
          f"Pos: {current_pos} | Return: {trade_return:+.4f} | Port: {portfolio_value:.2f}")
    with open(log_file, 'a') as f:
        f.write(f"{today.date()},{pred_momentum},{day_current_signal},{day_current_pos},{trade_return},{portfolio_value},{this_positon_return}\n")

# Letzte offene Position zum letzten Close schließen
if current_pos != 0 and entry_price is not None:
    last_close = aapl_close.loc[test_dates[-1]]
    final_return = current_pos * (last_close - entry_price) / entry_price
    portfolio_value *= (1 + final_return)
    portfolio_value *= (1 - FEE)
    print(f"\nFinal close | Return: {final_return:+.4f} | Port: {portfolio_value:.2f}€")

print(f"\n=== Endwert: {portfolio_value:.2f} (Start: 10000€, Rendite: {(portfolio_value/10000-1)*100:+.2f}%) ===")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Seed set to 1


Output()

/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


Output()

2024-01-02 | Pred: -0.00253 | Signal: 0 | Pos: 0 | Return: +0.0000 | Port: 10000.00


...

/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


Output()

2024-09-06 | Pred: 0.00474 | Signal: 0 | Pos: 0 | Return: +0.0000 | Port: 9857.39


/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


Output()